# Mayavi库的基本元素

## 1、Mayavi库中主要有两大部分功能:

### 1.1 用于处理图形可视化和图形操作的mlab模块.
mlab包含:
* 绘图函数：实现已有的数据进行可视化显示，可以是numpy数组构建的，也可以是外部读取的，比如读取一个文件。
* 图形控制函数：实质上是对mayavi中的figure进行控制，比如可以通过gcf获得当前视图的指针，也可以通过clf来清空当前图形，通过close关闭当前图形    * 图形修饰函数：对当前绘制的函数进行一定的修饰和装饰，比如说绘制完图形之后需要增加一个颜色标识栏。
* 图形控制函数，其他函数和Mlab管线控制函数或者是对坐标轴增加相应的标签等，比如使用xlaber可以在x轴上增加相应的标签
* 相机控制函数：对相机的操作，比如说move函数来移动相机到某个位置上，使用pitch,roll,yaw函数控制相机进行旋转等
* 其他函数：
    * animate:生成一段动态的可视化效果
    * get_engine:获得当前管线的engine
 
  
### 1.2 操作管线对象窗口对象的api.
Mlab管线控制：可以设置当前管线的数据源也可以为当前绘制管线增加数据集使用addataset等。

Mayavi API:
* 管线基础对象:可以通过这类函数获得Mayavi管线的各个基本对象,包括Scene,Source,Filter,ModuleManager,Module,PipelineBase,Engine等。
* 主视窗和UI对象：用于视图绘制和UI绘制的,比如MayaviScene,SceneEditor等。
* 我们可以在其他库中构建的用户界面里嵌入Mayavi窗口,比如可以结合TraitsUI和Mayavi构建一个可交互的三维可视化应用

mayavi的官方网址其中有所有的函数说明地址：
http://docs.enthought.com/mayavi/mayavi/index.html

In [7]:
%gui qt4
from mayavi import mlab


In [23]:
"""
快速绘图实例：
mayavi跟tvtk相比提供了更方便的可视化功能.通过调用mlab.mash绘制一个三维空间中图形的实例.
mayavi中mlab模块提供了方便快捷地绘制三维图发函数，只要把数据准备好，通常只需要调用一次mlab的函数就可以看到数据的三维可视化效果。
"""
x = [[-1,1,1,-1,-1],[-1,1,1,-1,-1]]
y = [[-1,-1,-1,-1,-1],[1,1,1,1,1]]
z = [[1,1,-1,-1,1],[1,1,-1,-1,1]]
s = mlab.mesh(x,y,z)


Tips:
按住shift键用左键对场景进行拖动平移场景或用shift+键盘键进行拖动
鼠标右键上下拖动或使用+-按键进行场景缩放
按ctrl键+左键拖动可以滚动相机,在程序中，通过传递关键字representation给mash函数可以指定绘制的表现形式.
如果默认的，是surface我们可以换成wireframe或points。

## 2、Mayavi管线的层级
### 2.1、最高层为 Engine:
引擎用于建立和销毁场景Scene，并不显示在管线的视图中.
* Scence场景:每个场景下有多个数据集合Sources，为Mayavi可视化提供了数据
* Filter:应用于Scence上对数据进行变换
* Module Manager:用来控制颜色，用来表示标量和矢量数据，
* 在pipline视图中表示为Colors and Legends节点。
* 可视化Modules:用于最终数据的显示，如线条，平面等

In [8]:
#可以通过mlab.show_pipline()或点击左侧的第一个按钮显示mayavi管线的对话框
#可以看见左侧用树状控件显示的mayavi一系列管线
#为深入了解mayavi管线，我们先注释掉最后一行通过命令行来了解管线中的对象
from numpy import pi,sin,cos,mgrid
from mayavi import mlab

#建立数据
dphi, dtheta = pi/250.0, pi/250.0
[phi,theta] = mgrid[0:pi+dphi*1.5:dphi,0:2*pi+dtheta*1.5:dtheta]
m0, m1, m2, m3, m4, m5, m6, m7 = 4, 3, 2, 3, 6, 2, 6, 4
r = sin(m0*phi)**m1 + cos(m2*phi)**m3 +sin(m4*theta)**m5 + cos(m6*theta)**m7
x = r*sin(phi)*cos(theta)
y = r*cos(phi)
z = r*sin(phi)*sin(theta)

print(x.shape,y.shape,z.shape)

#对数据进行三维可视化
s = mlab.mesh(x,y,z)


(252, 502) (252, 502) (252, 502)


## 2.2、管线中的对象scene

Mayavi Scene是最顶层的对象，表示场景，其配置界面中可以设置场景的背景，前景色，场景中的灯光及其他一些选项。

In [9]:
s = mlab.gcf() #获取对象的背景色（获取当前场景）
print(s) #输出当前的对象状态
print(s.scene.background)#输出当前场景的背景色

(0.5, 0.5, 0.5)


## 2.3、scene下一级:GridSource(网格数据源)
在配置界面中每一项为每个点所对应的标量数据的名称。

在本例中只有一个名为scalars的标量数据，它是传递给mash的数组，它表示mash表面的每个点的数据


In [10]:
source = s.children[0]#获取GridSource对象
print(source.name)#获取该节点名称
print(repr(source.data.points)) #GridSource的坐标后面length=多少就是坐标数组大小是多少
print(repr(source.data.point_data.scalars))#通过print函数打印出每个点对应的标量数组

GridSource
[(0.0, 2.0, 0.0), ..., (-0.025048897296365225, -1.9933803751132322, -0.00031479029697865414)], length = 126504
[0.0, ..., -0.00031479029697865414], length = 126504


## 2.4、GridSource下一级:PolydataNormals(数据源的法向量)

In [11]:
manager = source.children[0] #打印出PolydataNormals的相关信息
print(manager)


## 2.5、PolydataNormals下一级:Colors and legends

在colors and legends对象配置页面中,Scalar LUT选项卡可以设置将标量转化为颜色的查询表。

例如将LUT_mode改成Blues这样场景中的物体颜色会显示成白色到深蓝色的渐变。

如果勾选选项Show legerd选择框在场景中将添加一个颜色条来显示颜色和标量之间的关系,当然，也可以通过程序更改这两个选项。


In [12]:
colors = manager.children[0] #获取Colors and legends第一个子节点
colors.scalar_lut_manager.lut_mode = 'Blues'
colors.scalar_lut_manager.show_legend = True

## 2.6、Colors and legends下一级：surface对象

surface对象将GridSource输出的PolyData数据转换为最终在场景中显示的三维实体。

通过它的配置界面Actor选项卡可以对实体进行配置，例如我们将Reprentation设置成wireframe并将Line width设置为0，则实体最终采用细线框来显示。

我们将opasity设为0.6则实体变为半透明的状态，同样，我们也可以通过命令行的方式来更改这些属性。

In [13]:
surface = colors.children[0]#获取colors的第一个子节点
surface.actor.property.representation = 'wireframe'
surface.actor.property.opacity = 0.6

## 3、总结

程序配置属性的整个步骤可以描述为：
    1. 获得场景对象，mlab.gcf()
    
    2. 通过每个对象的children属性，在管线中找到需要修改的对象
    
    3.当其配置窗口中有多个选项卡的时候，或者有多个配置数组框时，属性需要一级一级获得。
    
    在程序配置属性的时候，为了方便配置各种属性的程序，mayavi提供了脚本录制功能。单机视图中的红色圆点，会打开一个脚本对话框，之后的界面配置操作都会被记录到此对话框中。